In [1]:
!git clone https://github.com/shekharkoirala/agentic_rag_workshop.git

Cloning into 'agentic_rag_workshop'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 0), reused 14 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 7.12 KiB | 1.02 MiB/s, done.


In [2]:
!cp -R agentic_rag_workshop/. .

In [ ]:
!pip install -r requirements.txt

### APIs


#### Weather API 
We will use the Weather API from `visualcrossing`. You have to generate you API key that you can later use to access this API. Follow the steps below: 

1. Signup up at https://www.visualcrossing.com/
2. Verify your account
3. Sign in and click on `Account` (blue button in the top right corner)
4. Under `Details` you should be able to see a `Key`
5. Copy the Key in `helper_functions/keys.py`


#### HuggingFace Token
We will use a model avaialble through a HuggingFace API. For that you need to generate a Token. Follow the steps below: 

1. Visit [HuggingFace](https://huggingface.co/) and sign up or log in.
2. Go to your profile (click your avatar), then "Settings" > "Access Tokens."
3. Click "New Token," select `Fine-grained` as Token Type role, and check the box `Make calls to the serverless Inference API`
4. Copy the Token in `helper_functions/keys.py`.


# OpenAI key 

Github MarketPlace Models : https://github.com/marketplace/models/azure-openai/gpt-4o-mini 

In [45]:
import io
import requests
from PIL import Image
import os
import sys

sys.path.append("./helper_functions")
from keys import WEATHER_KEY, HUGGING_FACE_KEY, OPENAI_KEY

from langchain import hub
from langchain.tools import StructuredTool
from langchain.pydantic_v1 import BaseModel, Field
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import AgentExecutor # execute agent
from langchain_openai import ChatOpenAI # call openAI as agent llm
from langchain.agents import create_tool_calling_agent # set up the agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [46]:
# verify wheather_key and other keys 
WEATHER_KEY

'JTMBMPKU2MALKW64DX9276HW7'

In [7]:
tool_input = """
is the water of liffey river in dublin  is salty ?
"""

# Run tool
wiki_tool.run(tool_input)

'Page: Blackrock, Dublin\nSummary: Blackrock (Irish: An Charraig Dhubh) is an affluent suburb of Dublin, Ireland, 3 km (1.9 mi) northwest of Dún Laoghaire. It is named after the local geological rock formation to be found in the area of Blackrock Park. In the late 18th century, the Blackrock Road was a common place for highway robberies. The Blackrock baths, provided for by the railway company in 1839, became popular in the 19th century but Blackrock is now a tourist destination.'

In [8]:
# define the function
def wikipedia_caller(query:str) ->str:
    """This function queries wikipedia through a search query."""
    return api_wrapper.run(query)

# Input parameter definition
class QueryInput(BaseModel):
    query: str = Field(description="Input search query")

# the tool description
description: str = (
        "A wrapper around Wikipedia. "
        "Useful for when you need to answer general questions about "
        "people, places, companies, facts, historical events, or other subjects. "
        "Input should be a search query."
    )


# fuse the function, input parameters and description into a tool.
my_own_wiki_tool = StructuredTool.from_function(
    func=wikipedia_caller,
    name="wikipedia",
    description=description,
    args_schema=QueryInput,
    return_direct=False,
)

# test the output of the tool
print(my_own_wiki_tool.run('Dublin'))

Page: Dublin
Summary: Dublin ( ; Irish: Baile Átha Cliath, pronounced [ˈbˠalʲə aːhə ˈclʲiə] or [ˌbʲlʲaː ˈclʲiə]) is the capital of Ireland. On a bay at the mouth of the River Liffey, it is in the province of Leinster, bordered on the south by the Dublin Mountains, a part of the Wicklow Mountains range. At the 2022 census, the city council area had a population of 592,713, while Dublin City and its suburbs had a population of 1,263,219, and County Dublin had a population of 1,501,500, making it  the largest city by size on the island of Ireland.
A settlement was established in the area by the Gaels during or before the 7th century, followed by the Vikings. As the Kingdom of Dublin grew, it became Ireland's principal settlement by the 12th century Anglo-Norman invasion of Ireland. The city expanded rapidly from the 17th century and was briefly the second largest in the British Empire and sixth largest in Western Europe after the Acts of Union in 1800. Following independence in 1922, Dubl

In [47]:
# define the function
def extract_city_weather(city:str)->str:

    # Build the API URL
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}?key={WEATHER_KEY}&unitGroup=metric"

    response = requests.get(url)
    print(response.content)

    # extract response
    if response.status_code == 200:
        data = response.json()
        current_temp = data['days'][0]['temp']
        output = f"Current temperature in {city}: {current_temp}°C"
    else:
        output = f"Error: {response.status_code}"

    return output

# Input parameter definition
class WeatherInput(BaseModel):
    city: str = Field(description="Input city")

# the tool description
description: str = (
        "A wrapper around Weather API. "
        "Useful for when you need to answer weather about "
        "temperature, weather, humidity, rain, sunny, storm, or climate subjects. "
        "Input should be a city."
    )

# fuse the function, input parameters and description into a tool.
my_weather_tool = StructuredTool.from_function(
    func = extract_city_weather,
    name = "weather",
    description=description,
    args_schema=WeatherInput,
    return_direct=False
)


In [48]:
print(my_weather_tool.run('Dublin'))

b'{"queryCost":1,"latitude":53.3481,"longitude":-6.24827,"resolvedAddress":"Dublin, Ireland","address":"Dublin","timezone":"Europe/Dublin","tzoffset":0.0,"description":"Similar temperatures continuing with a chance of rain tomorrow, Tuesday & Sunday & a chance of snow Saturday.","days":[{"datetime":"2024-11-17","datetimeEpoch":1731801600,"tempmax":9.1,"tempmin":6.0,"temp":7.2,"feelslikemax":7.1,"feelslikemin":2.1,"feelslike":5.0,"dew":5.7,"humidity":90.1,"precip":1.7,"precipprob":100.0,"precipcover":33.33,"preciptype":["rain"],"snow":0.0,"snowdepth":0.0,"windgust":35.6,"windspeed":21.7,"winddir":264.9,"pressure":1016.1,"cloudcover":93.3,"visibility":14.5,"solarradiation":8.8,"solarenergy":0.8,"uvindex":1.0,"severerisk":10.0,"sunrise":"07:54:34","sunriseEpoch":1731830074,"sunset":"16:25:01","sunsetEpoch":1731860701,"moonphase":0.55,"conditions":"Rain, Overcast","description":"Cloudy skies throughout the day with rain.","icon":"rain","stations":["F6683","EIDW","EIME"],"source":"comb","ho

In [57]:
def text_to_image(payload:str):

    # Call the text-to-image API with the provided palaod
    API_URL = "https://api-inference.huggingface.co/models/black-forest-labs/FLUX.1-dev"
    headers = {"Authorization": f"Bearer {HUGGING_FACE_KEY}"}

    def query(payload):
        response = requests.post(API_URL, headers=headers, json=payload)
        if response.status_code != 200:
          print(response.text)
        return response.content

    image_bytes = query({
        "inputs": payload,
    })

    # import pdb
    # pdb.set_trace()
    image = Image.open(io.BytesIO(image_bytes))

    # Resize the image
    new_size = (400, 400)  # Example new size (width, height)
    resized_image = image.resize(new_size)


    # Save the resized image to a file
    import os
    os.makedirs("./images",exist_ok=True)
    image_path = f'images/image_{payload.replace(" ", "_")}.jpg'
    resized_image.save(image_path)

    # Return the path to the saved image
    return f'{image_path} '


# Input parameter definition
class ImageInput(BaseModel):
    payload: str = Field(description="Input Text Query")


# the tool description
images_description: str = (
       "A wrapper around Text to Image Generation API",
       "Useful for when you need to generate images from text",
       "Input should be a text query"
    )

# fuse the function, input parameters and description into a tool.
my_image_tool = StructuredTool.from_function(
    func = text_to_image,
    name = "text_to_image",
    description=description,
    args_schema = ImageInput,
    return_direct=False
)

In [58]:
# Test the output of your Tool
print(my_image_tool.run('Dublin'))

# TODO: Try generating more ouputs

images/image_Dublin.jpg 


In [59]:
token = "" # replace this

# token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"


llm = ChatOpenAI(model= model_name,
                 temperature=0,
                 api_key=token,
                 openai_api_base=endpoint)

In [60]:

# Load Tools
tools = [my_own_wiki_tool, my_weather_tool, my_image_tool]


# With this you let the agent know what its purpose is.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a nice assistant"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Define the agent (load the LLM and the list of tools)
agent = create_tool_calling_agent(llm = llm, tools = tools, prompt = prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print("Your agent is ready.")


Your agent is ready.


In [61]:
question_3 = "What should I visit in Cliffs of Moher? Show me a photo"

print(f"Question 1: {question_3}")
agent_executor.invoke({"input": question_3})

Question 1: What should I visit in Cliffs of Moher? Show me a photo


> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Cliffs of Moher'}`


Page: Cliffs of Moher
Summary: The Cliffs of Moher (; Irish: Aillte an Mhothair) are sea cliffs located at the southwestern edge of the Burren region in County Clare, Ireland. They run for about 14 kilometres (9 miles).  At their southern end, they rise 120 metres (390 ft) above the Atlantic Ocean at Hag's Head, and, 8 kilometres (5 miles) to the north, they reach their maximum height of 214 metres (702 ft) just north of O'Brien's Tower, a round stone tower near the midpoint of the cliffs, built in 1835 by Sir Cornelius O'Brien, then continue at lower heights. The closest settlements are the villages of Liscannor 6 km (4 miles) to the south, and Doolin 7 km (4 miles) to the north.
From the cliffs, and from atop the tower, visitors can see the Aran Islands in Galway Bay, the Maumturks and Twelve Pins mountain ranges to t

{'input': 'What should I visit in Cliffs of Moher? Show me a photo',
 'output': "The Cliffs of Moher are stunning sea cliffs located in County Clare, Ireland, stretching about 14 kilometers (9 miles) along the southwestern edge of the Burren region. They rise dramatically above the Atlantic Ocean, reaching a maximum height of 214 meters (702 ft) just north of O'Brien's Tower, which was built in 1835. The cliffs offer breathtaking views of the Aran Islands, the Maumturks and Twelve Pins mountain ranges, and Loop Head.\n\nThe Cliffs of Moher are one of Ireland's most popular tourist attractions, drawing around 1.5 million visitors each year.\n\nHere is a photo of the Cliffs of Moher:\n\n![Cliffs of Moher](https://upload.wikimedia.org/wikipedia/commons/5/5e/Cliffs_of_Moher%2C_Ireland.jpg)"}

In [62]:
question_3 = "can capybara live with aligator ?"

print(f"Question 1: {question_3}")
agent_executor.invoke({"input": question_3})

Question 1: can capybara live with aligator ?


> Entering new AgentExecutor chain...
Capybaras and alligators are both found in similar habitats, such as wetlands and rivers in South America, but they are not suitable companions. Capybaras are social rodents that live in groups and are herbivorous, while alligators are carnivorous reptiles that can be aggressive and territorial.

In the wild, alligators may prey on capybaras, especially if the capybaras are young or vulnerable. Therefore, it is not advisable to keep capybaras and alligators together, as the alligator poses a significant threat to the safety of the capybara.

> Finished chain.


{'input': 'can capybara live with aligator ?',
 'output': 'Capybaras and alligators are both found in similar habitats, such as wetlands and rivers in South America, but they are not suitable companions. Capybaras are social rodents that live in groups and are herbivorous, while alligators are carnivorous reptiles that can be aggressive and territorial.\n\nIn the wild, alligators may prey on capybaras, especially if the capybaras are young or vulnerable. Therefore, it is not advisable to keep capybaras and alligators together, as the alligator poses a significant threat to the safety of the capybara.'}

In [63]:
question_3 = "How is the weather right now in New York ?"

print(f"Question 1: {question_3}")
agent_executor.invoke({"input": question_3})

Question 1: How is the weather right now in New York ?


> Entering new AgentExecutor chain...

Invoking: `weather` with `{'city': 'New York'}`


b'{"queryCost":1,"latitude":40.7146,"longitude":-74.0071,"resolvedAddress":"New York, NY, United States","address":"New York","timezone":"America/New_York","tzoffset":-5.0,"description":"Cooling down with a chance of rain Thursday & Friday.","days":[{"datetime":"2024-11-17","datetimeEpoch":1731819600,"tempmax":17.3,"tempmin":7.3,"temp":11.9,"feelslikemax":17.3,"feelslikemin":5.1,"feelslike":11.1,"dew":-2.4,"humidity":37.9,"precip":0.0,"precipprob":2.0,"precipcover":0.0,"preciptype":null,"snow":0.0,"snowdepth":0.0,"windgust":35.0,"windspeed":17.5,"winddir":306.9,"pressure":1016.3,"cloudcover":7.3,"visibility":16.2,"solarradiation":123.5,"solarenergy":10.5,"uvindex":5.0,"severerisk":10.0,"sunrise":"06:45:54","sunriseEpoch":1731843954,"sunset":"16:36:01","sunsetEpoch":1731879361,"moonphase":0.56,"conditions":"Clear","description":"Clear conditio

{'input': 'How is the weather right now in New York ?',
 'output': 'The current temperature in New York is 11.9°C.'}